In [1]:
flagval = True
flagtst = True
imgsz = 224

In [2]:
import pandas as pd 
import matplotlib.pyplot as plt 
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

!pip3 install Augmentor
from Augmentor.Operations import Distort

%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
labels_all = np.load('drive/MyDrive/mytrain/labels_all.npy', allow_pickle=True)
labels_trn = np.load('drive/MyDrive/mytrain/labels_trn.npy', allow_pickle=True)
labels_val = np.load('drive/MyDrive/mytrain/labels_val.npy', allow_pickle=True)


print(labels_all.shape)

labels_list = []
for label in np.unique(labels_all):
    labels_list.append(label)
labels_dict = dict()
for i, label in enumerate(labels_list):
    labels_dict[label] = i

for i in range(len(labels_all)):
    labels_all[i] = labels_dict[labels_all[i]]
for i in range(len(labels_trn)):
    labels_trn[i] = labels_dict[labels_trn[i]]
for i in range(len(labels_val)):
    labels_val[i] = labels_dict[labels_val[i]]

(166708,)


In [5]:
fname = 'drive/MyDrive/mytrain/pictblock.txt'
filee = open(fname, 'r')
blocklen = int(filee.read())
filee.close()
print(blocklen)

200000


In [6]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [7]:
class TrnDataset(Dataset):
    def __init__(
        self, 
        labels_trn = labels_trn, 
        labels_all = labels_all,
        flagval = flagval,
        blocklen = blocklen):
        super().__init__()
        self.labels = None
        self.fname = None
        if flagval:
            self.labels = labels_trn
            self.fname = 'drive/MyDrive/mytrain/images_trn'
        else:
            self.labels = labels_all
            self.fname = 'drive/MyDrive/mytrain/images_all'

        self.topil = transforms.ToPILImage()
        self.d = Distort(probability=0.5, grid_height=5, grid_width=5, magnitude=10)
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.RandomAffine(15),
            transforms.RandomPerspective(),
            transforms.Resize(imgsz),
            # transforms.Normalize(mean=0.0, std=1.0),
            # AddGaussianNoise(0.0, np.random.uniform(0.0,0.1)),
            transforms.Normalize(mean=0.0, std=1.0),
        ])
        self.loaded = -1
        self.images = None
        self.blocklen = blocklen
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        block = index // self.blocklen
        if block != self.loaded:
            self.images = np.load(
                self.fname + str(block) + '.npy', allow_pickle=True)
            self.loaded = block
        image = self.images[index % self.blocklen]

        sh = image.shape
        sh = (int(sh[0] / max(sh) * imgsz), int(sh[1] / max(sh) * imgsz))
        trans = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize(sh)
        ])
        image = trans(image)[0].numpy()
        result = np.zeros((imgsz, imgsz), dtype=np.float32)
        result[
            (imgsz - sh[0])//2:(imgsz - sh[0])//2 + image.shape[0],
            (imgsz - sh[1])//2:(imgsz - sh[1])//2 + image.shape[1]
        ] = 1 - image
        image = self.topil(result)
        image = self.d.perform_operation([image])[0]
        image = self.transform(image)

        label = self.labels[index]

        return image, label

class ValDataset(Dataset):
    def __init__(self, labels = labels_val, blocklen = blocklen):
        super().__init__()
        self.labels = labels
        self.transform = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize(mean=0.0, std=1.0)
             ])
        self.loaded = -1
        self.images = None
        self.blocklen = blocklen
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        block = index // self.blocklen
        if block != self.loaded:
            self.images = np.load('drive/MyDrive/mytrain/images_val' + str(block) + '.npy', allow_pickle=True)
            self.loaded = block
        image = self.images[index % self.blocklen]

        sh = image.shape
        sh = (int(sh[0] / max(sh) * imgsz), int(sh[1] / max(sh) * imgsz))
        trans = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize(sh)
        ])
        image = trans(image)[0].numpy()
        result = np.zeros((imgsz, imgsz), dtype=np.float32)
        result[
            (imgsz - sh[0])//2:(imgsz - sh[0])//2 + image.shape[0],
            (imgsz - sh[1])//2:(imgsz - sh[1])//2 + image.shape[1]
        ] = 1 - image
        image = self.transform(result)

        label = self.labels[index]

        return image, label

In [8]:
class TstDataset(Dataset):
    def __init__(self, blocklen = blocklen):
        super().__init__()
        self.len = 41428
        self.transform = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize(mean=0.0, std=1.0)
             ])
        self.loaded = -1
        self.images = None
        self.blocklen = blocklen
        
    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        block = index // self.blocklen
        if block != self.loaded:
            self.images = np.load(
                'drive/MyDrive/mytrain/images_tst' + str(block)
                + '.npy', allow_pickle=True)
            self.loaded = block
        image = self.images[index % self.blocklen]

        sh = image.shape
        sh = (int(sh[0] / max(sh) * imgsz), int(sh[1] / max(sh) * imgsz))
        trans = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize(sh)
        ])
        image = trans(image)[0].numpy()
        result = np.zeros((imgsz, imgsz), dtype=np.float32)
        result[
            (imgsz - sh[0])//2:(imgsz - sh[0])//2 + image.shape[0],
            (imgsz - sh[1])//2:(imgsz - sh[1])//2 + image.shape[1]
        ] = 1 - image
        image = self.transform(result)

        return image

In [9]:
train_data = TrnDataset()
valid_data = ValDataset()
test__data = TstDataset()
len(train_data), len(valid_data), len(test__data)

(158333, 8375, 41428)

In [10]:
num_epochs = 400
num_classes = len(np.unique(labels_all))
# batch_size = 30 local
batch_size = 50
learning_rate = 0.0001

In [11]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [12]:
train_loader = DataLoader(dataset = train_data, batch_size = batch_size, shuffle=False, num_workers=0)
valid_loader = DataLoader(dataset = valid_data, batch_size = batch_size // 2, shuffle=False, num_workers=0)
test__loader = DataLoader(dataset = test__data, batch_size = batch_size // 2, shuffle=False, num_workers=0)

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

In [14]:
model = models.resnet50()
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = nn.Linear(in_features=2048, out_features=500, bias=True)
print(model)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [15]:
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [16]:
import progressbar
from sklearn.metrics import accuracy_score

In [ ]:
# keeping-track-of-losses 
train_losses = []
valid_losses = []

for epoch in range(1, num_epochs + 1):
    model.train()

    train_loss = 0.0
    bar = progressbar.ProgressBar(
        max_value = len(train_data) // batch_size + 
        int(len(train_data)%batch_size!=0))
    i = 0
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device, dtype=torch.long)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        i += 1
        bar.update(i)
    bar.finish()
    train_loss = train_loss/len(train_loader.sampler)
    print(
        'Epoch: {} \tTraining Loss: {:.5f}'.format(epoch, train_loss), 
        end=' ')

    model.eval()

    if flagval:
        valid_loss = 0.0
        accuracy = 0
        for data, target in valid_loader:
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            accuracy += accuracy_score(
                output.cpu().argmax(axis=1), 
                target.cpu(), normalize=False)
            loss = criterion(output, target.long())
            valid_loss += loss.item() * data.size(0)
        valid_loss = valid_loss/len(valid_loader.sampler)
        accuracy /= len(valid_loader.sampler)
        print(
            '\tValidation Loss: {:.5f} \tAccuracy: {:.5f}'.format(
                valid_loss, accuracy))
    else:
        print()

    if flagtst:
        file = open(
            'drive/MyDrive/results/' + str(epoch) + '.csv', 'w')
        strans = 'Id,Category\n'
        currid = 0
        for data in test__loader:
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            preds = output.cpu().argmax(axis=1)
            for pred in preds:
                currid += 1
                strans += str(currid) + ',' + str(labels_list[pred]) + '\n'
        file.write(strans)
        file.close()

    #PATH = "saves/secondmodel_" + str(epoch)
    #torch.save(model.state_dict(), PATH)


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/functional.py:92: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  img = torch.from_numpy(np.array(pic, np.float32, copy=False))
/usr/local/lib/python3.6/dist-packages/torchvision/transforms/functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value is zero
  warnings.warn("Argument fill/fillcolor is not supported for Tensor input. Fill value is zero")
100% (3167 of 3167) |####################| Elapsed Time: 0:28:59 Time:  0:28:59


Epoch: 1 	Training Loss: 3.11295 	Validation Loss: 0.51319 	Accuracy: 0.85994


100% (3167 of 3167) |####################| Elapsed Time: 0:28:56 Time:  0:28:56


Epoch: 2 	Training Loss: 0.41116 	Validation Loss: 0.19835 	Accuracy: 0.94507


100% (3167 of 3167) |####################| Elapsed Time: 0:28:52 Time:  0:28:52


...

Epoch: 19 	Training Loss: 0.03919 	Validation Loss: 0.08786 	Accuracy: 0.98042


100% (3167 of 3167) |####################| Elapsed Time: 0:28:56 Time:  0:28:56


Epoch: 20 	Training Loss: 0.03821 	Validation Loss: 0.07331 	Accuracy: 0.98293


 72% (2288 of 3167) |##############      | Elapsed Time: 0:20:56 ETA:   0:08:03

Buffered data was truncated after reaching the output size limit.